<a href="https://colab.research.google.com/github/InowaR/python_seminars/blob/main/lesson12/seminar12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# 📌 Создайте класс-функцию, который считает факториал числа при вызове экземпляра.
# 📌 Экземпляр должен запоминать последние k значений.
# 📌 Параметр k передаётся при создании экземпляра.
# 📌 Добавьте метод для просмотра ранее вызываемых значений и их факториалов.
# 📌 Доработаем задачу 1. 📌 Создайте менеджер контекста, который при выходе сохраняет значения в JSON файл.

import json
import math
import sys


class FactorialCache:
  def __init__(self, k, filename):
    self.k = k
    self.cache = {}
    self.filename = filename

  def __call__(self, n):
    if n in self.cache:
      print("Взяли из кэша")
      return self.cache[n]
    if n < 0:
      raise ValueError("n должен быть неотрицательным")
    elif n == 0 or n == 1:
      result = 1
    else:
      result = n * self.__call__(n - 1)

    self.cache[n] = result
    print("Посчитали")
    return result

  def __enter__(self):
    return self

  def __exit__(self, exc_type, exc_val, exc_tb):
    if exc_type is not None:
      return

    with open(self.filename, "w") as f:
      json.dump(self.cache, f)

  def history(self):
    return list(self.cache.items())


# cache = FactorialCache(k=3, filename="factorial_cache.json")

# with cache as fc:
#   print(fc(5))
#   print(fc(5))
#   print(fc(5))

# print(cache.history())


# Создайте класс-генератор. 📌 Экземпляр класса должен генерировать факториал числа
# в диапазоне от start до stop с шагом step. 📌 Если переданы два параметра, считаем step=1.
# 📌 Если передан один параметр, также считаем start=1.


class FactorialGenerator:
  def __init__(self, start=1, stop=None, step=1):
    if stop is None:
      stop = start
      start = 1

    if step < 1:
      raise ValueError("Шаг должен быть больше 0")

    self.start = start
    self.stop = stop
    self.step = step
    self.n = start - step

  def __iter__(self):
    return self

  def __next__(self):
    if self.n >= self.stop:
      raise StopIteration
    self.n += self.step
    return math.factorial(self.n)


# for f in FactorialGenerator(5, 10):
#   print(f)

# print()

# for f in FactorialGenerator(10):
#   print(f)

# print()

# for f in FactorialGenerator(stop=5):
#   print(f)


class FactRange:
    def __init__(self, *args):
        data = args[:3]
        self.start, self.step = 1, 1
        if len(data) == 1:
            self.stop = data[0]
        elif len(data) == 2:
            self.start, self.stop = data
        else:
            self.start, self.stop, self.step = data
        self.data = [*range(self.start, self.stop+1, self.step)]
        self.value = 1

    def _get_fact(self, limit):
        fact = 1
        for i in range(1, limit + 1):
            fact *= i
        self.value = fact
        return fact

    def __iter__(self):
        return self

    def __next__(self):
        if self.data:
            return self._get_fact(self.data.pop(0))
        raise StopIteration

    def __str__(self):
        return str(self.value)


# if __name__ == '__main__':
#     factorial_gen = FactRange(5,10,3)
#     for i in factorial_gen:
#         print(i)


# 📌 Доработайте класс прямоугольник из прошлых семинаров.
# 📌 Добавьте возможность изменять длину и ширину прямоугольника
# и встройте контроль недопустимых значений (отрицательных).
# 📌 Используйте декораторы свойств.


class Rectangle:
    __slots__ = ("_length", "_width")

    def __init__(self, length, width=None):
        self._length = length
        self._width = width if width else length

    @property
    def length(self):
        return self._length

    @length.setter
    def length(self, value):
        if value < 0:
            raise ValueError("Длина не может быть отрицательной")
        self._length = value

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        if value < 0:
            raise ValueError("Ширина не может быть отрицательной")
        self._width = value

    def area(self):
        return self._length * self._width

    def perimeter(self):
        return 2 * (self._length + self._width)


# rect = Rectangle(5)

# print(f"Длина: {rect.length}")
# print(f"Ширина: {rect.width}")
# print(f"Площадь: {rect.area()}")
# print(f"Периметр: {rect.perimeter()}")

# print()

# rect.length = 10
# rect.width = 4

# print(f"Длина: {rect.length}")
# print(f"Ширина: {rect.width}")
# print(f"Площадь: {rect.area()}")
# print(f"Периметр: {rect.perimeter()}")

# print()
# print(f"Размер объекта: {sys.getsizeof(rect)} байт")


# 📌 Изменяем класс прямоугольника.
# 📌 Заменяем пару декораторов проверяющих длину и ширину на дескриптор с валидацией размера.


class SideValidate:

    def __init__(self, min_value: float = None, max_value: float = None):
        self.min_value = min_value
        self.max_value = max_value

    def __set_name__(self, owner, name):
        self.param_name = '_' + name

    def __get__(self, instance, owner):
        return getattr(instance, self.param_name)

    def __set__(self, instance, value):
        self.validate(value)
        setattr(instance, self.param_name, value)

    def validate(self, value):
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{value} is lesser than {self.min_value}')
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f'{value} is greater than {self.max_value}')


class Rectangle:
    side_a = SideValidate(10, 25)
    side_b = SideValidate(10, 25)

    def __init__(self, a, b):
        self.side_a = a
        self.side_b = b

    def __str__(self):
        return f'Прямоугольник {self.side_a} x {self.side_b}'


rect_1 = Rectangle(9, 20)
print(rect_1)

ValueError: 9 is lesser than 10